In [88]:
#creating a bangla tokenizer 


In [89]:
#training the llama model on the bangla dataset 

In [90]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# #download unsloth
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes datasets transformers


In [ ]:
# !pip install datasets tokenizers transformers[sentencepiece]

In [91]:
from datasets import load_dataset
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
import re

In [92]:
import wandb
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
from tokenizers import Tokenizer
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from huggingface_hub import HfApi, list_models
import torch
from unsloth import FastLanguageModel
from tqdm import tqdm
from huggingface_hub import login
from peft import get_peft_model, LoraConfig

In [93]:
# load bangla dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train")

#link to dataset  = https://huggingface.co/datasets/oscar-corpus/oscar

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [94]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [95]:
# BPE tokenizer 
tokenizer = Tokenizer(models.BPE())

In [96]:
# Set normalizer
tokenizer.normalizer = Sequence([
    NFD(),
    StripAccents(),
    Replace(Regex(r"[\s]+"), " "),
])

In [34]:
# Set normalizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [36]:
# Initialize the trainer
trainer = trainers.BpeTrainer(vocab_size=30000, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])


In [37]:
# Train the tokenizer
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# Save
tokenizer.save("bangla_tokenizer.json")

In [97]:
# load
bangla_tokenizer = Tokenizer.from_file("bangla_tokenizer.json")

In [98]:
# Wrap the tokenizer
bangla_tokenizer = PreTrainedTokenizerFast(tokenizer_object=bangla_tokenizer, unk_token="<unk>", pad_token="<pad>", cls_token="<s>", sep_token="</s>", mask_token="<mask>")

In [107]:
def format_dataset(examples):
    texts = examples["text"]
    eos_token_str = bangla_tokenizer.sep_token  
    # Tokenize the texts
    tokenized_texts = bangla_tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    return tokenized_texts


In [100]:
# Load and format the dataset
from datasets import load_dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train[:1%]")
dataset = dataset.map(format_dataset, batched=True)

In [101]:
# Initialize the model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.dev0.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.2+cu118. CUDA = 8.0. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.23.post1+cu118. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [102]:
# Attach LoRA Adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

In [103]:
# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bangla_tokenizer,
    mlm=False
)


In [104]:
training_args = TrainingArguments(
    output_dir="./bangla_llama",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=10_000,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="wandb",
    remove_unused_columns=False
)


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [105]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset
)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [106]:
# Train the model with progress bar
wandb.init(project="bangla-llama-finetuning")
for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch+1}/{training_args.num_train_epochs}")
    trainer.train()

# Save the model with progress bar
print("Saving the model...")
trainer.save_model("./bangla_llama")
bangla_tokenizer.save("bangla_llama")
print("Model saved.")

# Finish Weights & Biases run
wandb.finish()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,145 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 1
\        /    Total batch size = 4 | Total steps = 8,361
 "-____-"     Number of trainable parameters = 6,815,744


Epoch 1/3


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['id', 'text']